<a href="https://colab.research.google.com/github/com0040/offline_alpaco/blob/main/221028_%ED%85%8D%EC%8A%A4%ED%8A%B8_%EB%A7%88%EC%9D%B4%EB%8B%9D(%EB%A7%88%EB%AC%B4%EB%A6%AC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 주간웹툰 + 전체 한번에 만들기.ver 3
import requests # 서버 접근 라이브러리
from bs4 import BeautifulSoup # html에서 select을 사용하기 위해
import pandas as pd # dataframe / csv

web_info = { # 최종 딕셔너리
    'webtun_name' : [], # 웹툰 제목 저장 
    'JacGa' : [], # 작가 저장
    'star_score' : [], # 별점 저장 
    'week' : [] # 무슨 요일 웹툰인지 저장 
}

url = 'https://comic.naver.com/webtoon/weekdayList?week=mon' # 처음 페이지 url
resp = requests.get(url) # url 접근 승인 요구
soup = BeautifulSoup(resp.content, 'lxml') # html 형변환
a_tags = soup.select('ul.category_tab li a ') # 규칙과 묶음을 알아보고 될 수 있으면 하위를 다 적어둔다.

for idx, a_tag in enumerate(a_tags): # 총 몇번의 반복을 해야하는지 아는데 증감 규칙이 아닐때 태그에서 가져와서 사용
    if idx: # 첫값은 버린다 = 요일전체 / idx가 0으로 시작하는 것을 이용한다
        #(pop, remove -> 대상 명시해야해서 잘 안씀)

        # if idx == len(a_tags) - 1:
        #     week_kor =  a_tag.text[:2]
        # else:
        #     week_kor =  a_tag.text[:1]

        week_kor = a_tag.text[:2] if idx == len(a_tags) - 1 else a_tag.text[:1]
        
        week = a_tag['href'].split('=')[-1] # a_tag['herf']에있는 요일을 week에 저장(월~금, dayall)
        url = 'https://comic.naver.com/webtoon/weekdayList?week={}'.format(week) # week바꿔가며 url을 만듬
        # url = 'https://comic.naver.com/webtoon/weekdayList?week=' + week # week바꿔가며 url을 ----------- 방법 2
        resp = requests.get(url) # 접근
        soup = BeautifulSoup(resp.content, 'lxml') # 형 변환
        web_name = soup.select('ul.img_list li dl dt a') # 웹툰 제목 태그 모음 - 리스트
        jac_name = soup.select('ul.img_list li dl dd.desc a')  # 작가 이름 태그 모음 - 리스트
        score = soup.select('ul.img_list li dl dd div.rating_type strong') # 평점 태그 모음 - 리스트

        for a,b,c in zip(web_name,jac_name,score): # 요일별로 시행반복해야할 부분 
        # 웹툰, 작가, 평점 : 담긴 순서대로 진행 = 인덱스가 지정되므로 
        # web_name[i]['title'] <-> a.text a['title']
            web_info['webtun_name'].append(a.text) # 웹툰 이름에 태그의 텍스트를 불러서 저장
            web_info['JacGa'].append(b.text) # 작가에 태그의 텍스트를 불러서 저장 
            web_info['star_score'].append(c.text)# 별점 이름에 태그의 텍스트를 불러서 저장
            # web_info['webtun'].append(a['title'])
            # <-> 클래스 명으로 부를수 있으나 확인하고 편한 것으로 진행 
            web_info['week'].append(week_kor) # 무슨 요일 웹툰인지 저장

df = pd.DataFrame(web_info)
df

,webtun_name,JacGa,star_score,week
0,참교육,채용택 / 한가람,9.86,월
1,뷰티풀 군바리,설이 / 윤성원,9.81,월
2,퀘스트지상주의,박태준 만화회사,9.80,월
3,장씨세가 호위...,김인호 / 조형근,9.93,월
4,윈드브레이커,조용석,9.84,월
...,...,...,...,...
672,사파리 로맨스,yoshua / 윤정,9.67,매일
673,투 두 리스트,RYO. / FLEWing,9.77,매일
674,사생활 체크,리기 / 와치,9.71,매일
675,비타운,요신 / 김선우,8.08,매일


중간 중간 print() 해보면서 확인하는 습관 !!

In [ ]:
# 주간웹툰 + 전체 한번에 만들기.ver 4
import requests # 서버 접근 라이브러리
from bs4 import BeautifulSoup # html에서 select을 사용하기 위해
import pandas as pd # dataframe / csv
from tqdm.notebook import tqdm # for문 걸리는 시간 확인

web_info = { # 최종 딕셔너리
    '웹툰' : [], # 웹툰 제목 저장 
    '작가' : [], # 작가 저장
    '평점' : [], # 별점 저장 
    '요일' : [] # 무슨 요일 웹툰인지 저장 
}

url = 'https://comic.naver.com/webtoon/weekdayList?week=mon' # 처음 페이지 url
resp = requests.get(url) # url 접근 승인 요구
soup = BeautifulSoup(resp.content, 'lxml') # html 형변환
a_tags = soup.select('ul.category_tab li a ') # 반복할 요일
# 규칙과 묶음을 알아보고 될 수 있으면 하위를 다 적어둔다.

for idx, a_tag in tqdm(enumerate(a_tags)): # 총 몇번의 반복을 해야하는지 아는데 증감 규칙이 아닐때 태그에서 가져와서 사용
    if idx: # 첫값은 버린다 = 요일전체 / idx가 0으로 시작하는 것을 이용한다
        #(pop, remove -> 대상 명시해야해서 잘 안씀)
        # week_kor = a_tag.text[:2] if idx == len(a_tags) - 1 else a_tag.text[:1]a
        week_kor = a_tag.text[:-3]
        # week = a_tag['href'][49:]  ----- 방법 2
        # week = a_tag.replace('https://comic.naver.com/webtoon/weekdayList?week=','') ------------- 방법 3

        # a_tag['herf']에있는 요일을 week에 저장(월~금, dayall)
        url = url.split('=')[0] + '=' + a_tag['href'].split('=')[-1] # week바꿔가며 url을 ----------- 방법 2
        resp = requests.get(url) # 접근
        soup = BeautifulSoup(resp.content, 'lxml') # 형 변환
        web_name = soup.select('ul.img_list li dl dt a') # 웹툰 제목 태그 모음 - 리스트
        jac_name = soup.select('ul.img_list li dl dd.desc a')  # 작가 이름 태그 모음 - 리스트
        score = soup.select('ul.img_list li dl dd div.rating_type strong') # 평점 태그 모음 - 리스트

        for a,b,c in zip(web_name,jac_name,score): # 요일별로 시행반복해야할 부분 
        # 웹툰, 작가, 평점 : 담긴 순서대로 진행 = 인덱스가 지정되므로 
        # web_name[i]['title'] <-> a.text a['title']
            web_info['웹툰'].append(a.text) # 웹툰 이름에 태그의 텍스트를 불러서 저장
            web_info['작가'].append(b.text) # 작가에 태그의 텍스트를 불러서 저장 
            web_info['평점'].append(c.text)# 별점 이름에 태그의 텍스트를 불러서 저장
            # web_info['webtun'].append(a['title'])
            # <-> 클래스 명으로 부를수 있으나 확인하고 편한 것으로 진행 
            web_info['요일'].append(week_kor) # 무슨 요일 웹툰인지 저장

df_toon = pd.DataFrame(web_info)
df_toon

0it [00:00, ?it/s]

,웹툰,작가,평점,요일
0,참교육,채용택 / 한가람,9.86,월
1,뷰티풀 군바리,설이 / 윤성원,9.81,월
2,퀘스트지상주의,박태준 만화회사,9.80,월
3,장씨세가 호위...,김인호 / 조형근,9.93,월
4,윈드브레이커,조용석,9.84,월
...,...,...,...,...
672,사파리 로맨스,yoshua / 윤정,9.67,매일
673,투 두 리스트,RYO. / FLEWing,9.77,매일
674,사생활 체크,리기 / 와치,9.71,매일
675,비타운,요신 / 김선우,8.08,매일


In [ ]:
df_toon.to_csv('네이버웹툰요일별.csv',index = False)
naver_toons = pd.read_csv('네이버웹툰요일별.csv')
naver_toons

,웹툰,작가,평점,요일
0,참교육,채용택 / 한가람,9.86,월
1,뷰티풀 군바리,설이 / 윤성원,9.81,월
2,퀘스트지상주의,박태준 만화회사,9.80,월
3,장씨세가 호위...,김인호 / 조형근,9.93,월
4,윈드브레이커,조용석,9.84,월
...,...,...,...,...
672,사파리 로맨스,yoshua / 윤정,9.67,매일
673,투 두 리스트,RYO. / FLEWing,9.77,매일
674,사생활 체크,리기 / 와치,9.71,매일
675,비타운,요신 / 김선우,8.08,매일


In [ ]:
naver_toons.dtypes

웹툰     object
작가     object
평점    float64
요일     object
dtype: object

In [ ]:
naver_toons.groupby(['요일']).mean()['평점']

요일
금     9.820649
매일    9.556750
목     9.728837
수     9.830602
월     9.848289
일     9.882179
토     9.839744
화     9.793797
Name: 평점, dtype: float64

숙제) 장르를 나눠서 크롤링